<a href="https://colab.research.google.com/github/bobby-he/Neural_Tangent_Kernel/blob/master/notebooks/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Neural Tangent Kernel

Despite widespread practical success, the training behaviour of Deep Neural Networks (DNNs) has always been a poorly understood concept. Much focus has been placed on understanding convergence in parameter space, but it is well known that  the loss surface of DNNs are a high dimensional  and non-convex. They contain many saddle-points, valleys and symmetries that render it unreasonable to expect that a gradient based optimiser will be able to converge to the global minimum, if one exists.  In spite of this, DNNs are also known for their good generalisation properties, despite their seemingly obvious over-parametrisation. 

This report will study some recent papers that aim to shed light on the dyanmics of DNN training. The main idea involved is to view the training process in terms of the function $f_{\theta}$ that the DNN represents instead of the parameters $\theta$